<a href="https://colab.research.google.com/github/RyuichiSaito1/multilingual-economic-narratives/blob/main/notebooks/x_api_free_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* X API: https://docs.x.com/x-api/introduction
* Search Query: https://docs.x.com/x-api/posts/search/integrate/build-a-query
* X Dashboard: https://developer.x.com/en/portal/dashboard

In [2]:
#!/usr/bin/env python3
"""
Free tier: 100 reads/month, 500 writes/month, basic v2 endpoints
"""

import requests
import json
from datetime import datetime

try:
    import requests
except ImportError:
    !pip install requests

In [2]:
class SimpleXAPITester:
    def __init__(self, bearer_token: str):
        self.bearer_token = bearer_token
        self.headers = {"Authorization": f"Bearer {bearer_token}"}
        self.base_url = "https://api.twitter.com/2/tweets/search/recent"
        self.request_count = 0

    def search_tweets(self, query: str, lang: str = None, max_results: int = 10):

        search_query = query
        if lang:
            search_query += f" lang:{lang}"

        params = {
            "query": search_query,
            "max_results": min(max_results, 100),
            "tweet.fields": "created_at,lang,public_metrics"
        }

        print(f"Query: '{query}'")
        if lang:
            print(f"Language: {lang}")
        print(f"Full search query: '{search_query}'")
        print(f"Searching...")

        try:
            response = requests.get(self.base_url, headers=self.headers, params=params)
            self.request_count += 1
            print(f"Status: {response.status_code}")

            if response.status_code == 200:
                data = response.json()
                self.display_results(data, query, lang)
                return data
            else:
                print(f"Error: {response.text}")
                return None

        except Exception as e:
            print(f"Request failed: {e}")
            return None

    def display_results(self, data, query, lang):
        if "data" not in data or not data["data"]:
            print("No tweets found")
            return

        tweets = data["data"]
        print(f"\nFound {len(tweets)} tweets")

        languages = {}
        for tweet in tweets:
            tweet_lang = tweet.get("lang", "unknown")
            languages[tweet_lang] = languages.get(tweet_lang, 0) + 1

        print(f"Languages: {languages}")

        print("\nAll tweets:")
        for i, tweet in enumerate(tweets, 1):
            text = tweet.get("text", "")
            tweet_lang = tweet.get("lang", "N/A")
            likes = tweet.get("public_metrics", {}).get("like_count", 0)
            retweets = tweet.get("public_metrics", {}).get("retweet_count", 0)
            replies = tweet.get("public_metrics", {}).get("reply_count", 0)

            # Parse and format the creation date/time
            created_at_str = tweet.get("created_at", "")
            if created_at_str:
                try:
                    # Parse ISO format datetime from Twitter API
                    created_at = date_parser.parse(created_at_str)
                    # Format to a more readable format
                    formatted_date = created_at.strftime("%Y-%m-%d %H:%M:%S UTC")
                except:
                    formatted_date = created_at_str  # Fallback to raw string
            else:
                formatted_date = "Date not available"

            print(f"\n{i}. [{tweet_lang}] {text}")
            print(f"   Published: {formatted_date}")
            print(f"   Likes: {likes} | Retweets: {retweets} | Replies: {replies}")

        print(f"\nAPI requests used: {self.request_count}/100 (monthly limit)")

In [3]:
def main():

    BEARER_TOKEN = ""


    # Initialize tester
    tester = SimpleXAPITester(BEARER_TOKEN)

    # Economic keywords search in Finnish using OR logic
    # (price OR cost OR inflation OR deflation OR expensive OR cheap OR purchase OR sale)
    economic_keywords_query = "(hinta OR kustannus OR inflaatio OR deflaatio OR kallis OR halpa OR osto OR myynti)"

    tester.search_tweets(
        query=economic_keywords_query,
        lang="fi",
        max_results=10
    )

    print(f"\nTotal API requests: {tester.request_count}")

if __name__ == "__main__":
    main()

Query: '(hinta OR kustannus OR inflaatio OR deflaatio OR kallis OR halpa OR osto OR myynti)'
Language: fi
Full search query: '(hinta OR kustannus OR inflaatio OR deflaatio OR kallis OR halpa OR osto OR myynti) lang:fi'
Searching...
Status: 200

Found 10 tweets
Languages: {'fi': 10}

All tweets:

1. [fi] Vapaapäivä.  Heti aamusta kävin puolukassa. 🤭 Paikkana toripistefi. 30e 10 litraa puhdistettua eilen poimittua puolukkaa. Minusta ei ole paha hinta. https://t.co/xXQ1DnFUUC
   Published: 2025-09-22T06:40:08.000Z
   Likes: 0 | Retweets: 0 | Replies: 0

2. [fi] RT @PiiaMetshonkal3: Vuonna 2023 ilman huoltajaa alaikäisiä turvapaikanhakijoita saapui Suomeen 348 (ei ukrainalaisia mukana). Jos kaikki a…
   Published: 2025-09-22T06:39:37.000Z
   Likes: 0 | Retweets: 32 | Replies: 0

3. [fi] @PatrizioLaina &gt; "Inflaatio saatiin kuriin, valtion budjetti kääntyi ylijäämäiseksi, ja pörssikurssit nousivat. Yrityksillä ja kansalaisilla alkoi hiljalleen olla taas rahkeita kuluttaa."

Kansassa on to

In [3]:
class SimpleXAPITester:
    def __init__(self, bearer_token: str):
        self.bearer_token = bearer_token
        self.headers = {"Authorization": f"Bearer {bearer_token}"}
        self.base_url = "https://api.twitter.com/2/tweets/search/recent"
        self.request_count = 0

    def search_tweets(self, query: str, lang: str = None, max_results: int = 10, place_country: str = None):

        search_query = query
        if lang:
            search_query += f" lang:{lang}"

        if place_country:
            # Check if place_country is already in the query to avoid duplication
            if "place_country:" not in search_query:
                search_query += f" place_country:{place_country}"

        params = {
            "query": search_query,
            "max_results": min(max_results, 100),
            "tweet.fields": "created_at,lang,public_metrics"
        }

        print(f"Query: '{query}'")
        if lang:
            print(f"Language: {lang}")
        if place_country:
            print(f"Place Country: {place_country}")
            print("Note: place_country requires elevated access and may not work with basic/free tier")
        print(f"Full search query: '{search_query}'")
        print(f"Searching...")

        try:
            response = requests.get(self.base_url, headers=self.headers, params=params)
            self.request_count += 1
            print(f"Status: {response.status_code}")

            if response.status_code == 200:
                data = response.json()
                self.display_results(data, query, lang, place_country)
                return data
            else:
                error_text = response.text
                print(f"Error: {error_text}")

                # Check if it's a place_country operator error
                if "place_country" in error_text and "not available" in error_text:
                    print("Tip: The place_country operator requires elevated Twitter API access.")
                    print("Consider upgrading your API access level or removing the place_country parameter.")

                return None

        except Exception as e:
            print(f"Request failed: {e}")
            return None

    def display_results(self, data, query, lang, place_country=None):
        if "data" not in data or not data["data"]:
            print("No tweets found")
            return

        tweets = data["data"]
        print(f"\nFound {len(tweets)} tweets")

        languages = {}
        for tweet in tweets:
            tweet_lang = tweet.get("lang", "unknown")
            languages[tweet_lang] = languages.get(tweet_lang, 0) + 1

        print(f"Languages: {languages}")

        if place_country:
            print(f"Filtered by country: {place_country}")

        print("\nAll tweets:")
        for i, tweet in enumerate(tweets, 1):
            text = tweet.get("text", "")
            tweet_lang = tweet.get("lang", "N/A")
            likes = tweet.get("public_metrics", {}).get("like_count", 0)
            retweets = tweet.get("public_metrics", {}).get("retweet_count", 0)
            replies = tweet.get("public_metrics", {}).get("reply_count", 0)

            # Parse and format the creation date/time
            created_at_str = tweet.get("created_at", "")
            if created_at_str:
                try:
                    if date_parser:
                        # Parse ISO format datetime from Twitter API
                        created_at = date_parser.parse(created_at_str)
                        # Format to a more readable format
                        formatted_date = created_at.strftime("%Y-%m-%d %H:%M:%S UTC")
                    else:
                        # Fallback without dateutil
                        formatted_date = created_at_str.replace('T', ' ').replace('Z', ' UTC')
                except:
                    formatted_date = created_at_str  # Fallback to raw string
            else:
                formatted_date = "Date not available"

            print(f"\n{i}. [{tweet_lang}] {text}")
            print(f"   Published: {formatted_date}")
            print(f"   Likes: {likes} | Retweets: {retweets} | Replies: {replies}")

        print(f"\nAPI requests used: {self.request_count}/100 (monthly limit)")


In [4]:
def main():
    # Note: Replace with your actual bearer token
    BEARER_TOKEN = ""

    # Initialize tester
    tester = SimpleXAPITester(BEARER_TOKEN)

    # Economic keywords search in Finnish using OR logic
    economic_keywords_query = "(hinta OR kustannus OR inflaatio OR deflaatio OR kallis OR halpa OR osto OR myynti)"

    # Example 1: Search with place_country (requires elevated access)
    print("=== Search with place_country filtering ===")
    tester.search_tweets(
        query=economic_keywords_query,
        lang="fi",
        place_country="FI",
        max_results=10
    )

    print("\n" + "="*50 + "\n")

    # Example 2: Search without place_country (should work with free tier)
    print("=== Search without geographic filtering ===")
    tester.search_tweets(
        query=economic_keywords_query,
        lang="fi",
        max_results=10
    )

    print(f"\nTotal API requests: {tester.request_count}")

if __name__ == "__main__":
    main()

=== Search with place_country filtering ===
Query: '(hinta OR kustannus OR inflaatio OR deflaatio OR kallis OR halpa OR osto OR myynti)'
Language: fi
Place Country: FI
Note: place_country requires elevated access and may not work with basic/free tier
Full search query: '(hinta OR kustannus OR inflaatio OR deflaatio OR kallis OR halpa OR osto OR myynti) lang:fi place_country:FI'
Searching...
Status: 400
Error: {"errors":[{"parameters":{"query":["(hinta OR kustannus OR inflaatio OR deflaatio OR kallis OR halpa OR osto OR myynti) lang:fi place_country:FI"]},"message":"There were errors processing your request: Reference to invalid operator 'place_country'. Operator is not available in current product or product packaging. Please refer to complete available operator list at http://t.co/operators. (at position 93)"}],"title":"Invalid Request","detail":"One or more parameters to your request was invalid.","type":"https://api.twitter.com/2/problems/invalid-request"}
Tip: The place_country ope